# Environment set-up

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pathlib
path = pathlib.Path('/content/gdrive/MyDrive/Fyp')
try:
    path.mkdir()
    %cd '/content/gdrive/MyDrive/Fyp'
    ! git clone https://github.com/TanShengRong/FYP-RPI-RF-Sensing
    %cd './FYP-RPI-RF-Sensing/dataset/'
    print('Project folder created in your GoogleDrive')
except FileExistsError:
    %cd '/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing/'
    ! git pull
    %cd './dataset_v2/'

/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing
Already up to date.
/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing/dataset_v2


In [3]:
pwd

'/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing/dataset_v2'

In [4]:
ls

gestureA/  gestureC/  gestureE/


# Deep Learning

In [5]:
import pandas as pd
import numpy as np
import torch
import glob
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pdb
import os
import torchvision
import time

## Dataloading

In [7]:
#=== Params ===#
DATASET_DIR = '/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing/dataset_v2/'
GESTURE_DEGREE = "degree0"
DATA_ROW_LOWER_LIMIT = 0
DATA_ROW_UPPER_LIMIT = 460 #540 # 12s -> 600 (3 transitions)
BATCH_SIZE = 30
NUM_WORKERS = 0 # 2; 0 is default
SHUFFLE = True
# cols = [i for i in range(138)]
cols = [i for i in range(141)]
VALIDATION_SPLIT = .2
RANDOM_SEED = 42

In [8]:
torch.cuda.empty_cache()
torch.manual_seed(RANDOM_SEED)

In [9]:
#=== Dataloading ===#
import os
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader


class GestureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        # self.root_dir = '/content/gdrive/MyDrive/Fyp/FYP-RPI-RF-Sensing/dataset/'
        self.gesture_list = self.getGestureList()
        self.gesturePerFolder = 100 # change this if increasing no. of sample per gesture per degree
        self.gestureDegree = GESTURE_DEGREE
        self.transform = transform
    
    def __len__(self):
        return len(self.gesture_list) * self.gesturePerFolder
        
    def __getitem__(self, idx):
        item_gesture_type_idx = idx // self.gesturePerFolder
        item_gesture_type = self.gesture_list[item_gesture_type_idx],
        item_idx = idx % self.gesturePerFolder
        # print(item_gesture_type_idx, item_gesture_type[0], item_idx)
        item_gesture_path = os.path.join(
            self.root_dir,   
            item_gesture_type[0],
            self.gestureDegree,
            str(item_idx+1) + ".csv"
            )
        item_gesture_df = pd.read_csv(item_gesture_path, header=0) 
        # real = item_gesture_df.iloc[ DATA_ROW_LOWER_LIMIT:DATA_ROW_UPPER_LIMIT, :138 ]
        real = item_gesture_df.iloc[ DATA_ROW_LOWER_LIMIT:DATA_ROW_UPPER_LIMIT, :141 ]
        real.columns = cols
        # imag = item_gesture_df.iloc[ DATA_ROW_LOWER_LIMIT:DATA_ROW_UPPER_LIMIT, 138:277-1 ] * 1j
        imag = item_gesture_df.iloc[ DATA_ROW_LOWER_LIMIT:DATA_ROW_UPPER_LIMIT, 141:282 ] * 1j
        imag.columns = cols
        comp = abs(real.add(imag, fill_value=(0)))
        
        features = comp.values
        features = torch.FloatTensor(features)
        features = features.view(1, DATA_ROW_UPPER_LIMIT-DATA_ROW_LOWER_LIMIT, item_gesture_df.shape[1]//2)
        
        targets = np.zeros(len(self.gesture_list))
        targets[item_gesture_type_idx] = 1
        targets = torch.FloatTensor(targets)

        return features, targets
        
    def getGestureList(self):
        gestureList = []
        for _, dirnames, filenames in os.walk(self.root_dir):
            if any("gesture" in dirname for dirname in dirnames):
                gestureList += dirnames
        print(gestureList)
        return gestureList

In [10]:
from torch.utils.data.sampler import SubsetRandomSampler

#=== stratified sampling ===#
dataset = GestureDataset(DATASET_DIR)
dataset_size = len(dataset)
no_of_gesture = len(dataset.gesture_list)

indicesA = list(range(0, dataset_size//3))
indicesC = list(range(dataset_size//3, dataset_size//3*2))
indicesE = list(range(dataset_size//3*2, dataset_size))

split = int(np.floor(VALIDATION_SPLIT * dataset_size//3))
print(split)
if SHUFFLE :
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indicesA)
    np.random.shuffle(indicesC)
    np.random.shuffle(indicesE)
train_indicesA, val_indicesA = indicesA[split:], indicesA[:split]
train_indicesC, val_indicesC = indicesC[split:], indicesC[:split]
train_indicesE, val_indicesE = indicesE[split:], indicesE[:split]

train_indices = train_indicesA + train_indicesC + train_indicesE
val_indices = val_indicesA + val_indicesC + val_indicesE

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, 
                          sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=BATCH_SIZE,
                               sampler=valid_sampler)

['gestureA', 'gestureC', 'gestureE']
20


In [11]:
for i in enumerate(train_loader):
    print(i[0], i[1][0].shape)
for i in enumerate(validation_loader):
    print(i[0], i[1][0].shape)

0 torch.Size([30, 1, 460, 141])
1 torch.Size([30, 1, 460, 141])
2 torch.Size([30, 1, 460, 141])
3 torch.Size([30, 1, 460, 141])
4 torch.Size([30, 1, 460, 141])
5 torch.Size([30, 1, 460, 141])
6 torch.Size([30, 1, 460, 141])
7 torch.Size([30, 1, 460, 141])
0 torch.Size([30, 1, 460, 141])
1 torch.Size([30, 1, 460, 141])


# ResNet


In [12]:
import torch
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.8),
    nn.Linear(2048, 3)
)
model.eval()


Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
for batch_idx, (data, target) in enumerate(train_loader):
    with torch.no_grad():
        output = model(data.repeat(1, 3, 1, 1))
        # print('output', output)
        # print('target', target)
    break

In [14]:
output.shape

torch.Size([30, 3])

In [15]:
for batch_idx, (data, target) in enumerate(validation_loader):
    with torch.no_grad():
        output = model(data.repeat(1, 3, 1, 1))
        print('output', output)
        print('target', target)
        _, label = torch.max(target, 1)
        _, predicted = torch.max(output, 1)
        print('label', label)
        print('predicted', predicted)
    break

output tensor([[-0.0357,  0.0874,  0.0374],
        [-0.0370,  0.0861,  0.0390],
        [-0.0349,  0.0887,  0.0359],
        [-0.0360,  0.0873,  0.0384],
        [-0.0362,  0.0853,  0.0386],
        [-0.0359,  0.0864,  0.0389],
        [-0.0356,  0.0880,  0.0382],
        [-0.0364,  0.0864,  0.0386],
        [-0.0364,  0.0870,  0.0384],
        [-0.0368,  0.0861,  0.0388],
        [-0.0357,  0.0865,  0.0383],
        [-0.0363,  0.0869,  0.0385],
        [-0.0364,  0.0870,  0.0381],
        [-0.0361,  0.0866,  0.0389],
        [-0.0364,  0.0851,  0.0387],
        [-0.0363,  0.0862,  0.0380],
        [-0.0360,  0.0868,  0.0385],
        [-0.0358,  0.0875,  0.0366],
        [-0.0358,  0.0869,  0.0381],
        [-0.0363,  0.0855,  0.0389],
        [-0.0360,  0.0875,  0.0374],
        [-0.0359,  0.0861,  0.0388],
        [-0.0364,  0.0856,  0.0381],
        [-0.0366,  0.0861,  0.0389],
        [-0.0370,  0.0856,  0.0390],
        [-0.0358,  0.0865,  0.0388],
        [-0.0363,  0.0867,  0.0

In [15]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [16]:
for epoch in range(10):
    total=0
    correct=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        if torch.cuda.is_available():
            data=data.cuda()
            target=target.cuda()

        optimizer.zero_grad()
        output = model(data.repeat(1, 3, 1, 1))
        # print('output', output)
        # print('target', target)

        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()

        #accuracy
        # _, predicted = torch.max(output.data, 1)
        # total += target.size(0)
        # correct += (predicted == target).sum().item()
        _, label = torch.max(target, 1)
        _, predicted = torch.max(output, 1)
        comparison_tensor = torch.eq(label, predicted).long()
        correct += torch.count_nonzero(comparison_tensor)
        total += comparison_tensor.shape[0]

    
    print('Train Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))
    torch.save(model.state_dict(), '/content/gdrive/My Drive/Fyp/FYP-RPI-RF-Sensing/dataset/1903resNet.pth')
    
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_loader):
            if torch.cuda.is_available():
                data=data.cuda()
                target=target.cuda()

            output = model(data.repeat(1, 3, 1, 1))
            loss = criterion(output, torch.max(target, 1)[1])
            # _, predicted = torch.max(output.data, 1)
            # total += target.size(0)
            # correct += (predicted == target).sum().item()
            _, label = torch.max(target, 1)
            _, predicted = torch.max(output, 1)
            print('label', label)
            print('predicted', predicted)
            comparison_tensor = torch.eq(label, predicted).long()
            correct += torch.count_nonzero(comparison_tensor)
            total += comparison_tensor.shape[0]
    # print('Test : Loss: {:.3f}, Acc:{:.3f}\n'.format(loss, correct / len(validation_loader.dataset)))
    print('Test Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))

Train Epoch: 0  Loss: 0.485| Acc:0.579
label tensor([2, 1, 1, 2, 0, 1, 1, 0, 2, 2, 2, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 2, 2, 2])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
label tensor([2, 2, 0, 1, 2, 1, 0, 1, 1, 2, 2, 1, 0, 1, 2, 0, 2, 0, 0, 2, 0, 1, 0, 0,
        0, 1, 1, 2, 0, 2])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
Test Epoch: 0  Loss: 1.297| Acc:0.333
Train Epoch: 1  Loss: 0.140| Acc:0.950
label tensor([1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 2,
        1, 1, 2, 0, 0, 1])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
label tensor([0, 2, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2, 2, 1, 0, 2,
        0, 2, 1, 1, 1, 2])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
model.load_state_dict(torch.load('/content/gdrive/My Drive/Fyp/FYP-RPI-RF-Sensing/dataset/1903resNet.pth'))

<All keys matched successfully>

In [19]:
for epoch in range(10, 20):
    total=0
    correct=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        if torch.cuda.is_available():
            data=data.cuda()
            target=target.cuda()

        optimizer.zero_grad()
        output = model(data.repeat(1, 3, 1, 1))
        # print('output', output)
        # print('target', target)

        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()

        #accuracy
        # _, predicted = torch.max(output.data, 1)
        # total += target.size(0)
        # correct += (predicted == target).sum().item()
        _, label = torch.max(target, 1)
        _, predicted = torch.max(output, 1)
        comparison_tensor = torch.eq(label, predicted).long()
        correct += torch.count_nonzero(comparison_tensor)
        total += comparison_tensor.shape[0]

    
    print('Train Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))
    torch.save(model.state_dict(), '/content/gdrive/My Drive/Fyp/FYP-RPI-RF-Sensing/dataset/1903resNet.pth')
    
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_loader):
            if torch.cuda.is_available():
                data=data.cuda()
                target=target.cuda()

            output = model(data.repeat(1, 3, 1, 1))
            loss = criterion(output, torch.max(target, 1)[1])
            # _, predicted = torch.max(output.data, 1)
            # total += target.size(0)
            # correct += (predicted == target).sum().item()
            _, label = torch.max(target, 1)
            _, predicted = torch.max(output, 1)
            print('label', label)
            print('predicted', predicted)
            comparison_tensor = torch.eq(label, predicted).long()
            correct += torch.count_nonzero(comparison_tensor)
            total += comparison_tensor.shape[0]
    # print('Test : Loss: {:.3f}, Acc:{:.3f}\n'.format(loss, correct / len(validation_loader.dataset)))
    print('Test Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))

Train Epoch: 11  Loss: 0.008| Acc:1.000
label tensor([0, 1, 2, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 0, 2, 1, 2, 0,
        1, 1, 2, 0, 0, 2])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0])
label tensor([2, 1, 1, 1, 1, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 2, 0, 0, 0, 2, 2, 1, 0, 2,
        2, 0, 2, 1, 1, 0])
predicted tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2,
        0, 0, 0, 0, 0, 0])
Test Epoch: 11  Loss: 1.833| Acc:0.450
Train Epoch: 12  Loss: 0.001| Acc:1.000
label tensor([1, 0, 0, 2, 1, 0, 0, 1, 0, 2, 2, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 1,
        2, 2, 1, 2, 0, 0])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        2, 2, 0, 0, 0, 0])
label tensor([0, 1, 0, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1,
        2, 1, 0, 1, 1, 0])
predicted tensor([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
model.load_state_dict(torch.load('/content/gdrive/My Drive/Fyp/FYP-RPI-RF-Sensing/dataset/1903resNet.pth'))

<All keys matched successfully>

In [16]:
for epoch in range(20, 30):
    total=0
    correct=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        if torch.cuda.is_available():
            data=data.cuda()
            target=target.cuda()

        optimizer.zero_grad()
        output = model(data.repeat(1, 3, 1, 1))
        # print('output', output)
        # print('target', target)

        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()

        #accuracy
        # _, predicted = torch.max(output.data, 1)
        # total += target.size(0)
        # correct += (predicted == target).sum().item()
        _, label = torch.max(target, 1)
        _, predicted = torch.max(output, 1)
        comparison_tensor = torch.eq(label, predicted).long()
        correct += torch.count_nonzero(comparison_tensor)
        total += comparison_tensor.shape[0]

    
    print('Train Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))
    torch.save(model.state_dict(), '/content/gdrive/My Drive/Fyp/FYP-RPI-RF-Sensing/dataset/1903resNet.pth')
    
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_loader):
            if torch.cuda.is_available():
                data=data.cuda()
                target=target.cuda()

            output = model(data.repeat(1, 3, 1, 1))
            loss = criterion(output, torch.max(target, 1)[1])
            # _, predicted = torch.max(output.data, 1)
            # total += target.size(0)
            # correct += (predicted == target).sum().item()
            _, label = torch.max(target, 1)
            _, predicted = torch.max(output, 1)
            print('label', label)
            print('predicted', predicted)
            comparison_tensor = torch.eq(label, predicted).long()
            correct += torch.count_nonzero(comparison_tensor)
            total += comparison_tensor.shape[0]
    # print('Test : Loss: {:.3f}, Acc:{:.3f}\n'.format(loss, correct / len(validation_loader.dataset)))
    print('Test Epoch: {}  Loss: {:.3f}| Acc:{:.3f}'.format(epoch+1, loss.item(),correct / total))

Train Epoch: 21  Loss: 0.157| Acc:0.983
label tensor([2, 2, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 1, 2, 0, 2, 0, 1, 2, 0, 0, 0, 0, 1,
        0, 0, 1, 2, 0, 1])
predicted tensor([0, 2, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 1, 2, 0, 2, 0, 1, 2, 0, 0, 0, 0, 1,
        0, 0, 1, 2, 0, 1])
label tensor([2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 2, 2,
        2, 1, 1, 2, 1, 1])
predicted tensor([2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 2, 2,
        2, 1, 1, 2, 1, 1])
Test Epoch: 21  Loss: 0.160| Acc:0.967
Train Epoch: 22  Loss: 0.004| Acc:0.996
label tensor([2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2,
        0, 1, 1, 0, 1, 2])
predicted tensor([2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2,
        0, 1, 1, 0, 1, 2])
label tensor([1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0,
        0, 2, 0, 2, 1, 2])
predicted tensor([1, 1, 1, 0, 2, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 0, 0, 